In [ ]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from utils import time_vector, circular_task_reference, plot_tracking

plt.style.use('seaborn-v0_8')


# Control cinemático del manipulador RRR planar


## 1. Parámetros


In [ ]:
params_rrr = {
    'L1': 0.7,
    'L2': 0.6,
    'L3': 0.4
}

sim_config_rrr = {
    'kp': np.diag([9.0, 9.0, 6.0]),
    't0': 0.0,
    'tf': 10.0,
    'n_steps': 800
}

t_rrr, dt_rrr = time_vector(sim_config_rrr)


## 2. Cinemática directa


In [ ]:
def fkine_rrr(q, params):
    q1, q2, q3 = q
    L1, L2, L3 = params['L1'], params['L2'], params['L3']
    c1, s1 = np.cos(q1), np.sin(q1)
    c12, s12 = np.cos(q1 + q2), np.sin(q1 + q2)
    c123, s123 = np.cos(q1 + q2 + q3), np.sin(q1 + q2 + q3)

    x = L1 * c1 + L2 * c12 + L3 * c123
    y = L1 * s1 + L2 * s12 + L3 * s123
    theta = q1 + q2 + q3
    return np.array([x, y, theta])

fkine_rrr(np.zeros(3), params_rrr)


## 3. Jacobiano


In [ ]:
def jacobian_rrr(q, params):
    q1, q2, q3 = q
    L1, L2, L3 = params['L1'], params['L2'], params['L3']

    s1 = np.sin(q1)
    c1 = np.cos(q1)
    s12 = np.sin(q1 + q2)
    c12 = np.cos(q1 + q2)
    s123 = np.sin(q1 + q2 + q3)
    c123 = np.cos(q1 + q2 + q3)

    J = np.array([
        [-L1 * s1 - L2 * s12 - L3 * s123, -L2 * s12 - L3 * s123, -L3 * s123],
        [ L1 * c1 + L2 * c12 + L3 * c123,  L2 * c12 + L3 * c123,  L3 * c123],
        [1.0, 1.0, 1.0]
    ])
    return J

jacobian_rrr(np.zeros(3), params_rrr)


### Derivación simbólica


In [ ]:
q1_s, q2_s, q3_s, L1_s, L2_s, L3_s = sp.symbols('q1 q2 q3 L1 L2 L3')
px = L1_s * sp.cos(q1_s) + L2_s * sp.cos(q1_s + q2_s) + L3_s * sp.cos(q1_s + q2_s + q3_s)
py = L1_s * sp.sin(q1_s) + L2_s * sp.sin(q1_s + q2_s) + L3_s * sp.sin(q1_s + q2_s + q3_s)
pt = q1_s + q2_s + q3_s
J_symbolic = sp.Matrix([px, py, pt]).jacobian([q1_s, q2_s, q3_s])
sp.simplify(J_symbolic)


## 4. Trayectoria objetivo


In [ ]:
radius_rrr = 0.25
center_rrr = (1.1, 0.2)
omega_rrr = 0.7

Xd_planar_rrr, Vd_planar_rrr = circular_task_reference(radius_rrr, center_rrr, omega_rrr, t_rrr)
theta_d_rrr = np.unwrap(np.arctan2(Vd_planar_rrr[1], Vd_planar_rrr[0]))
theta_dp_rrr = np.gradient(theta_d_rrr, t_rrr)

Xd_rrr = np.vstack((Xd_planar_rrr, theta_d_rrr))
Vd_rrr = np.vstack((Vd_planar_rrr, theta_dp_rrr))
Xd_rrr[:, :3]


## 5. Control cinemático


In [ ]:
def kinematic_control_step_rrr(q, params, xd, xdp, kp):
    x = fkine_rrr(q, params)
    J = jacobian_rrr(q, params)
    dq = np.linalg.solve(J, xdp + kp @ (xd - x))
    return dq, x

kinematic_control_step_rrr(np.zeros(3), params_rrr, Xd_rrr[:, 0], Vd_rrr[:, 0], sim_config_rrr['kp'])


## 6. Simulación


In [ ]:
def simulate_rrr_control(q0, params, Xd, Vd, t, kp):
    n = t.size
    dt = t[1] - t[0]
    q = np.zeros((3, n))
    X = np.zeros((3, n))
    q[:, 0] = q0
    X[:, 0] = fkine_rrr(q0, params)
    for k in range(1, n):
        dq, _ = kinematic_control_step_rrr(q[:, k-1], params, Xd[:, k-1], Vd[:, k-1], kp)
        q[:, k] = q[:, k-1] + dt * dq
        X[:, k] = fkine_rrr(q[:, k])
    return q, X

q0_rrr = np.array([0.1, -0.2, 0.3])
theta_rrr, X_rrr = simulate_rrr_control(q0_rrr, params_rrr, Xd_rrr, Vd_rrr, t_rrr, sim_config_rrr['kp'])
X_rrr[:, :3]


In [ ]:
theta_error_rrr = np.angle(np.exp(1j * (Xd_rrr[2] - X_rrr[2])))
position_error_rrr = np.linalg.norm(Xd_rrr[:2] - X_rrr[:2], axis=0)
max_pos_error_rrr = float(np.max(position_error_rrr))
max_theta_error_rrr = float(np.max(np.abs(theta_error_rrr)))
print(f'Error máximo de posición: {max_pos_error_rrr:.4f} m')
print(f'Error máximo de orientación: {max_theta_error_rrr:.4f} rad')
assert max_pos_error_rrr < 5e-2
assert max_theta_error_rrr < 8e-2


## 7. Visualización


In [ ]:
fig_rrr = plot_tracking(X_rrr[:2], Xd_rrr[:2], t_rrr)
fig_rrr.axes[0].set_title('Trayectoria planar (x, y)')
fig_rrr.axes[1].set_title('Errores de posición en el plano')
fig_rrr


In [ ]:
plt.figure(figsize=(6, 3))
plt.plot(t_rrr, np.rad2deg(theta_error_rrr), label='Error de orientación')
plt.xlabel('Tiempo [s]')
plt.ylabel('Error [deg]')
plt.title('Seguimiento de la orientación θ')
plt.legend()
plt.grid(True)
plt.tight_layout()


### Animación


In [ ]:
def animate_rrr(theta, X, Xd, params, skip=2):
    L1, L2, L3 = params['L1'], params['L2'], params['L3']

    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_aspect('equal', adjustable='box')
    margin = 0.3
    x_all = np.concatenate([X[0], Xd[0]])
    y_all = np.concatenate([X[1], Xd[1]])
    ax.set_xlim(np.min(x_all) - margin, np.max(x_all) + margin)
    ax.set_ylim(np.min(y_all) - margin, np.max(y_all) + margin)
    ax.set_xlabel('x [m]')
    ax.set_ylabel('y [m]')
    ax.set_title('Robot planar RRR')

    desired_line, = ax.plot(Xd[0], Xd[1], '--', linewidth=1.2, label='Trayectoria deseada')
    trail_line,   = ax.plot([], [], ':', linewidth=1.5, label='Rastro')
    link1_line, = ax.plot([], [], 'k-', linewidth=3)
    link2_line, = ax.plot([], [], 'k-', linewidth=3)
    link3_line, = ax.plot([], [], 'k-', linewidth=3)
    ee_point,   = ax.plot([], [], 'o', markersize=5)
    ax.legend(loc='upper right')

    def joint_positions(q):
        q1, q2, q3 = q
        p0 = np.array([0.0, 0.0])
        p1 = p0 + np.array([L1 * np.cos(q1), L1 * np.sin(q1)])
        p2 = p1 + np.array([L2 * np.cos(q1 + q2), L2 * np.sin(q1 + q2)])
        p3 = p2 + np.array([L3 * np.cos(q1 + q2 + q3), L3 * np.sin(q1 + q2 + q3)])
        return p0, p1, p2, p3

    trail_x, trail_y = [], []

    def update(frame):
        k = frame * skip
        if k >= theta.shape[1]:
            k = theta.shape[1] - 1
        q = theta[:, k]
        p0, p1, p2, p3 = joint_positions(q)
        link1_line.set_data([p0[0], p1[0]], [p0[1], p1[1]])
        link2_line.set_data([p1[0], p2[0]], [p1[1], p2[1]])
        link3_line.set_data([p2[0], p3[0]], [p2[1], p3[1]])
        ee_point.set_data(p3[0], p3[1])
        trail_x.append(p3[0])
        trail_y.append(p3[1])
        trail_line.set_data(trail_x, trail_y)
        return link1_line, link2_line, link3_line, ee_point, trail_line

    frames = max(1, theta.shape[1] // skip)
    anim = FuncAnimation(fig, update, frames=frames, interval=40, blit=True, repeat=False)
    plt.close(fig)
    return anim

anim_rrr = animate_rrr(theta_rrr, X_rrr, Xd_rrr, params_rrr)
HTML(anim_rrr.to_jshtml())
